# See how many points are inside and outside of the labels, before and after filtering

In [1048]:
# Libraries
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path

In [1049]:
# List of paths to dataset
BASE_DIR = '../../data/eval_data'
ARCS_DIR = 'arcs'
ARCS_FILTERED_DIR = 'arcs_azimuth_filtered'
ARCS_LABEL_FILTERED_DIR = 'arcs_label_filtered'
ARCS_LABEL_DIR = 'labels'

In [1050]:
# Evaluate filter (before eval script)
BASE_DIR = '../../data'
ARCS_DIR = 'velodyne_points'
ARCS_FILTERED_DIR = 'filtered_points'
ARCS_LABEL_FILTERED_DIR = 'label_filtered_points'
ARCS_LABEL_DIR = 'pillar_pred_format_labels'

In [1051]:
# test_dir = 'test_folder'

In [1052]:
def point_is_in_an_roi_box(point, roi_box_list):
    # Extract the point coordinates
    x, y, z = point
    
    for bbox in roi_box_list:
        # Extract the box parameters
        # dz, dy, dx, cx, cy, cz, yaw = bbox
        cx, cy, cz, dx, dy, dz, yaw = bbox
        
        # Create the rotation matrix for the yaw angle
        cos_yaw = np.cos(-yaw)  # Negative for the inverse rotation
        sin_yaw = np.sin(-yaw)
        
        rotation_matrix = np.array([
            [cos_yaw, -sin_yaw, 0],
            [sin_yaw,  cos_yaw, 0],
            [0,       0,       1]
        ])
        
        # Translate the point to the box's coordinate frame
        translated_point = np.array([x - cx, y - cy, z - cz])
        
        # Rotate the point to align with the box
        rotated_point = rotation_matrix.dot(translated_point)
        
        # Check if the point is within the box dimensions
        half_lengths = [dz / 2, dy / 2, dx / 2]
        in_box = all([
            -half_lengths[i] <= rotated_point[i] <= half_lengths[i] for i in range(3)
        ])
    
        if in_box:
            return True
    
    return False

In [1053]:
def get_roi_boxes(label_file):
    bboxes = []
    with open(label_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = [float(value) for value in parts[8:15]]  # Extract the bounding box dimensions and location            
            bboxes.append(bbox)
    return np.array(bboxes)

In [1054]:
def is_label_point(point, label_file_path):
    list_roi_boxes = get_roi_boxes(label_file_path)
    if point_is_in_an_roi_box(point, list_roi_boxes):
        return True
    return False

In [1055]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [1056]:
def get_frame_point_statistics(data_file_path, label_file_path):
    # Running totals
    total_points = 0
    label_points = 0

    points_df = convert_to_dataframe(data_file_path)
    
    # For each point in the data
    for index, row in points_df.iterrows():
    # for index, row in points_df.iloc[:100].iterrows():
        # Increment the total
        total_points += 1
        point = (row['x'], row['y'], row['z'])
        # If the point is a label point
        if is_label_point(point, label_file_path):
            # Increment the label total
            label_points += 1

    return total_points, label_points

In [1057]:
# test_files = Path(BASE_DIR, test_dir)
test_dir = Path(BASE_DIR, ARCS_FILTERED_DIR)
files = [f for f in os.listdir(test_dir) if f.endswith('.bin')]
random.shuffle(files)
files = files[:1000]
print(files)

['002975.bin', '005248.bin', '000719.bin', '005838.bin', '003671.bin', '001141.bin', '000711.bin', '006105.bin', '000238.bin', '002529.bin', '000144.bin', '004685.bin', '004960.bin', '004437.bin', '002234.bin', '006224.bin', '002589.bin', '001380.bin', '002920.bin', '002961.bin', '004056.bin', '002047.bin', '001902.bin', '005599.bin', '002763.bin', '001022.bin', '004996.bin', '002946.bin', '000532.bin', '001839.bin', '001361.bin', '003754.bin', '002654.bin', '000867.bin', '005508.bin', '004586.bin', '002823.bin', '000138.bin', '006438.bin', '006291.bin', '004033.bin', '001656.bin', '004233.bin', '001284.bin', '002356.bin', '002503.bin', '001486.bin', '000367.bin', '003812.bin', '001252.bin', '003213.bin', '002732.bin', '000536.bin', '002070.bin', '006107.bin', '002407.bin', '001020.bin', '004121.bin', '002693.bin', '004122.bin', '002598.bin', '000691.bin', '003711.bin', '005845.bin', '005415.bin', '005741.bin', '002874.bin', '005605.bin', '002566.bin', '001017.bin', '000237.bin', '0060

In [1058]:
def get_dataset_point_statistics(data_path, label_path):
    # Running totals
    num_frames = 0
    total_num_points = 0
    total_num_label_points = 0
    total_file_size = 0
    
    for filename in files:
        # Get file index
        file_id, extension = os.path.splitext(filename)
        print(file_id, end=' ')
        
        # Append file name to locations
        data_file_path = Path(data_path, filename)
        label_filename = file_id + '.txt'
        label_file_path = Path(label_path, label_filename)

        # Get the number of points, and the number of points inside the labels
        num_frame_points, num_frame_label_points = get_frame_point_statistics(data_file_path, label_file_path)

        # Get the file size
        size = os.path.getsize(data_file_path)
        
        # Add to running total
        num_frames += 1
        total_num_points += num_frame_points
        total_num_label_points += num_frame_label_points
        total_file_size += size

    # return total_num_points / num_frames, total_num_label_points / num_frames
    return num_frames, total_num_points, total_num_label_points, total_file_size

In [1059]:
def evaluate_filter(dir, dataset_name):
    # Create directory path
    data_path = Path(BASE_DIR, dir)
    label_path = Path(BASE_DIR, ARCS_LABEL_DIR)
    print(data_path)

    # Get the dataset statistics
    num_frames, total_num_points, total_num_label_points, total_file_size = get_dataset_point_statistics(data_path, label_path)

    results = {
        'dataset_name': dataset_name,
        'num_frames': num_frames,
        'total_num_points': total_num_points,
        'total_num_label_points': total_num_label_points,
        'total_num_non_label_points': total_num_points - total_num_label_points,
        'avg_frame_pts': total_num_points / num_frames,
        'avg_frame_label_pts': total_num_label_points / num_frames,
        'avg_frame_non_label_pts': (total_num_points - total_num_label_points) / num_frames,
        'total_files_size': total_file_size
    }

    return results

### Run datasets through filter evaluator, print results

In [ ]:
results = evaluate_filter(ARCS_DIR, 'ARCS')

..\..\data\velodyne_points
002975 005248 000719 005838 003671 001141 000711 006105 000238 002529 000144 004685 004960 004437 002234 006224 002589 001380 002920 002961 004056 002047 001902 005599 002763 001022 004996 002946 000532 001839 001361 003754 002654 000867 005508 004586 002823 000138 006438 006291 004033 001656 004233 001284 002356 002503 001486 000367 003812 001252 003213 002732 000536 002070 006107 002407 001020 004121 002693 004122 002598 000691 003711 005845 005415 005741 002874 005605 002566 001017 000237 006008 003689 000122 003466 000788 002454 003870 005769 000833 003846 002511 004136 000242 002522 000265 005419 000530 004738 002101 004818 002788 001417 000728 006075 000136 004656 004799 006299 002773 006172 001576 004728 001879 002130 004625 001672 002726 003863 004011 001169 001903 000772 002923 001818 005474 001744 005276 001498 005960 002341 000018 004719 005819 005515 004402 005681 000017 000026 000447 004439 005463 000795 003177 002546 002109 003073 001114 005835 

In [ ]:
filter_results = evaluate_filter(ARCS_FILTERED_DIR, 'Filtered ARCS')

In [ ]:
label_filter_results = evaluate_filter(ARCS_LABEL_FILTERED_DIR, 'Label Filtered ARCS')

In [ ]:
results['percent_of_non_label_pts_retained'] = results['total_num_non_label_points'] / results['total_num_non_label_points']
results['percent_of_label_pts_retained'] = results['total_num_label_points'] / results['total_num_label_points']
results['percent_of_original_file_size'] = results['total_files_size'] / results['total_files_size']

In [ ]:
filter_results['percent_of_non_label_pts_retained'] = filter_results['total_num_non_label_points'] / results['total_num_non_label_points']
filter_results['percent_of_label_pts_retained'] = filter_results['total_num_label_points'] / results['total_num_label_points']
filter_results['percent_of_original_file_size'] = filter_results['total_files_size'] / results['total_files_size']

In [ ]:
label_filter_results['percent_of_non_label_pts_retained'] = label_filter_results['total_num_non_label_points'] / results['total_num_non_label_points']
label_filter_results['percent_of_label_pts_retained'] = label_filter_results['total_num_label_points'] / results['total_num_label_points']
label_filter_results['percent_of_original_file_size'] = label_filter_results['total_files_size'] / results['total_files_size']

In [ ]:
results_df = pd.DataFrame([results, filter_results, label_filter_results])
# results_df = pd.DataFrame([results, filter_results])
columns_to_display = ['dataset_name',
                      'avg_frame_non_label_pts',
                      'avg_frame_label_pts',
                      'percent_of_non_label_pts_retained',
                      'percent_of_label_pts_retained',
                      'percent_of_original_file_size']
display(results_df[columns_to_display])

In [ ]:
columns_to_display = ['dataset_name',
                      'percent_of_non_label_pts_retained',
                      'percent_of_label_pts_retained',
                      'percent_of_original_file_size']
display(results_df[columns_to_display])

In [ ]:
# test_dir = '../../data/eval_data/test_folder'
# results = evaluate_filter(test_dir, 'test')
# print(results)

In [ ]:
import matplotlib.pyplot as plt
# These tests were done with azimuth_resolution = 0.01, height_resolution = 0.25

# Defining the parameter changes and their corresponding results
parameters = ['+0.2', '0', '-0.1', '-0.2', '-0.5', '-1']
non_label_points_retained = [0.888438, 0.690376, 0.195896, 0.165541, 0.142986, 0.136502]
label_points_retained = [0.965184, 0.933963, 0.88492, 0.874891, 0.855174, 0.837658]

# Creating the plot
plt.figure(figsize=(10, 5))
plt.plot(parameters, non_label_points_retained, label='Non-label Points Retained', marker='o')
plt.plot(parameters, label_points_retained, label='Label Points Retained', marker='o')
plt.title('Effect of Distance Parameter Adjustment on Point Retention')
plt.xlabel('Parameter Adjustment')
plt.ylabel('Retention Rate')
plt.legend()
plt.grid(True)
plt.show()